# NAIP Wrangle

In [1]:
# Load variables from the previous notebook
%store -r den_tract_cdc_gdf den_census_tracts_data_dir dropped_joined_den_tracts_gdf

In [2]:
# Import libraries to help with ...

# Reproducible file paths
import os # Reproducible file paths
from glob import glob # Find files by pattern
import pathlib # Find the home folder
import time # formatting time
import warnings # Filter warning messages
import zipfile # Work with zip files
from io import BytesIO # Stream binary (zip) files

# Find files by pattern
import numpy as np # adjust images 
import matplotlib.pyplot as plt # Overlay pandas and xarry plots, Overlay raster and vector data
import requests # Request data over HTTP

# Work with tabular, vector, and raster data
import cartopy.crs as ccrs # CRSs (Coordinate Reference Systems)
import geopandas as gpd # work with vector data
import geoviews as gv # holoviews extension for data visualization
import hvplot.pandas # Interactive tabular and vector data
import hvplot.xarray # Interactive raster
import pandas as pd # Group and aggregate
import pystac_client # Modify returns from API
import shapely # Perform geometric operations on spatial data
import xarray as xr # Adjust images
import rioxarray as rxr # Work with geospatial raster data
from rioxarray.merge import merge_arrays # Merge rasters

# Processing and regression related
from scipy.ndimage import convolve # Image and signal processing
from sklearn.model_selection import KFold # Cross validation
from scipy.ndimage import label # Labels connected features in an array
from sklearn.linear_model import LinearRegression # Work with linear regression models
from sklearn.model_selection import train_test_split # Split data into subsets - evaluate model
from tqdm.notebook import tqdm # Visualize progress of iterative operations

# import to be able to save plots
import holoviews as hv # be able to save hvplots

# Suppress third party warnings - 'ignore'
warnings.simplefilter('ignore')

# Prevent GDAL from quitting due to momentary disruptions
os.environ["GDAL_HTTP_MAX_RETRY"] = "5"
os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"

# Get Data URLs

In [3]:
# Connect to the planetary computer catalog
e84_catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
e84_catalog.title

'Microsoft Planetary Computer STAC API'

In [6]:
# Convert geometry to lat/lon for STAC
den_tract_latlon_gdf = den_tract_cdc_gdf.to_crs(4326)

# Define a path to save NDVI stats
den_ndvi_stats_path = os.path.join(den_census_tracts_data_dir, 'denver-ndvi-stats.csv')

# Check for existing data - do not access duplicate tracts

# Create a list to accumulate downloaded tracts
den_downloaded_tracts = []
# If Else statement to control specifc blocks of code
# Code to execute if the condition is true
if os.path.exists(den_ndvi_stats_path):
    den_ndvi_stats_df = pd.read_csv(den_ndvi_stats_path)
    den_downloaded_tracts = den_ndvi_stats_df.tract.values
# Code to execute if the condition is false
else:
    print('No census tracts downloaded so far')


# Loop through each census tract

# Create list of dataframes, list needs to be outside of loop
den_scene_dfs = []
# Start for loop
for i, tract_values in tqdm(den_tract_latlon_gdf.iterrows()):
    den_tract = tract_values.tract2010
    # Check if statistics are already downloaded for this tract
    if not (den_tract in den_downloaded_tracts):
        # Retry up to 5 times in case of a momentary disruption
        i = 0
        retry_limit = 5
        # Loop for executing code block as long as specified condition is true
        while i < retry_limit:
            # Try accessing the STAC
            try:
                # Search for tiles
                naip_search = e84_catalog.search(
                    # In the NAIP collection
                    collections=["naip"],
                    # That intersect with geometry of census tracts
                    intersects=shapely.to_geojson(tract_values.geometry),
                    # In the year 2021
                    datetime="2020"
                )
                
                # Build dataframe with tracts and tile urls
                den_scene_dfs.append(pd.DataFrame(dict(
                    tract=den_tract,
                    # Convert datetime value to a pandas Timestamp, then extract just the date 
                    date=[pd.to_datetime(scene.datetime).date() 
                          # of the items in the NAIP search
                          for scene in naip_search.items()],
                    # Reference image from the assets folder of the items in the NAIP search?   
                    rgbir_href=[scene.assets['image'].href for scene in naip_search.items()],
                )))
                # Add break to prevent long waits during debugging
                break
            # Try again in case of an APIError
            except pystac_client.exceptions.APIError:
                print(
                    f'Could not connect with STAC server. '
                    f'Retrying tract {den_tract}...')
                time.sleep(2)
                i += 1
                # Skip the rest of the current iteration and move on to the next one
                continue
    
# Concatenate the url dataframes
# Code to execute if the condition is true
if den_scene_dfs:
    den_scene_df = pd.concat(den_scene_dfs).reset_index(drop=True)
# Code to execute if the condition is false
else:
    den_scene_df = None

# Preview the URL DataFrame
den_scene_df

No census tracts downloaded so far


0it [00:00, ?it/s]

KeyboardInterrupt: 

# Compute NDVI Stats

In [ ]:
# Skip this step if data are already downloaded 
if not den_scene_df is None:
    # Get an example tract
    den_ex_tract = dropped_joined_chi_tracts_gdf.loc[0].tract2010
    # Get example gdf based off of the example tract
    den_ex_scene_gdf = den_scene_df[den_scene_df.tract==den_ex_tract]

    # Loop through all images for tract
    den_tile_das = []
    # Create for loop, iterate over rows
    for _, href_s in den_ex_scene_gdf.iterrows():
        # Open vsi connection to data
        den_tile_da = rxr.open_rasterio(
            # File path/url to the multispectral image
            href_s.rgbir_href, 
            # Create masked array, then remove any single-dimensional axes from it
            masked=True).squeeze()
        
        # Crop data to the bounding box of the census tract
        # Create the boundary
        den_boundary = (
            # Using the census tract gdf
            den_tract_cdc_gdf
            # Set the 'tract2010' as the index of the gdf
            .set_index('tract2010')
            # Select the ex_tract from that gdf
            .loc[[den_ex_tract]]
            # Set to the same CRS as the images for the tract
            .to_crs(den_tile_da.rio.crs)
            # Access the geometry of this tract to perform further operations on it
            .geometry
        )
        # Crop the data to bounding box
        den_crop_da = den_tile_da.rio.clip_box(
            # Compute bounding box (min and max coordinates) of census tract geometry
            *den_boundary.envelope.total_bounds,
            # Expand bounding box slightly beyond its original extent to ensure full coverage
            auto_expand=True)

        # Clip data to the boundary of the census tract
        den_clip_da = den_crop_da.rio.clip(den_boundary, all_touched=True)

        # Compute NDVI ((NIR - Red)/(NIR + Red))
        ndvi_da = (
            (den_clip_da.sel(band=4) - den_clip_da.sel(band=1)) 
            / (den_clip_da.sel(band=4) + den_clip_da.sel(band=1))
        )

        # Accumulate result
        den_tile_das.append(ndvi_da)

    # Merge data
    den_scene_da = merge_arrays(den_tile_das)

    # Mask vegetation
    den_veg_mask = (den_scene_da>.3)

    # Calculate mean patch size
    # Label the connected areas
    labeled_patches, num_patches = label(den_veg_mask)
    # Count number of non-negative occurances, flatten the array to 1D
    patch_sizes = np.bincount(labeled_patches.ravel())[1:]
    # Get the mean 
    mean_patch_size = patch_sizes.mean()

    # Calculate edge density
    kernel = np.array([
        [1, 1, 1], 
        [1, -8, 1], 
        [1, 1, 1]])
    # Apply convolution to the vegetation mask
    edges = convolve(
        # Input array - the array to apply convolution on
        den_veg_mask, 
        # Kernel array - the smaller array that defines the filter to be applied
        kernel, 
        # Input array is extended beyond its boundaries by - 
        # filling all values beyond the edge with the same constant value
        mode='constant')
    
    # Calculate edge density = 
    # count of edge pixels present / total number of pixels in the veg_mask
    den_edge_density = np.sum(edges != 0) / den_veg_mask.size

# Repeat for all tracts

In [ ]:
# Skip this step if data are already downloaded 
if not den_scene_df is None:
    all_den_ndvi_dfs = []
    # Loop through the census tracts with URLs
    for tract, tract_date_gdf in tqdm(den_scene_df.groupby('tract')):
        # Open all images for tract
        all_den_tile_das = []
        # Create for loop, iterate over rows
        for _, href_s in tract_date_gdf.iterrows():
            # Open vsi connection to data
            all_den_tile_da = rxr.open_rasterio(
                # File path/url to the multispectral image
                href_s.rgbir_href, 
                # Create masked array, then remove any single-dimensional axes from it
                masked=True).squeeze()
            
            # Crop data to the bounding box of the census tract
            # Create the boundary
            all_den_boundary = (
                # Using the census tract gdf
                den_tract_cdc_gdf
                # Set the 'tract2010' as the index of the gdf
                .set_index('tract2010')
                # Select the tracts from the gdf
                .loc[[tract]]
                # Set to the same CRS as the images for the tracts
                .to_crs(all_den_tile_da.rio.crs)
                # Access the geometry of the tracts to perform further operations
                .geometry
            )
            # Crop the data to bounding box
            all_den_crop_da = all_den_tile_da.rio.clip_box(
                # Compute bounding box (min and max coordinates) of census tract geometry
                *all_den_boundary.envelope.total_bounds,
                # Expand bounding box slightly beyond its original extent to ensure full coverage
                auto_expand=True)
            
            # Clip data to the boundary of the census tract
            all_den_clip_da = all_den_crop_da.rio.clip(all_den_boundary, all_touched=True)

            # Compute NDVI ((NIR - Red)/(NIR + Red))
            all_den_ndvi_da = (
                (all_den_clip_da.sel(band=4) - all_den_clip_da.sel(band=1)) 
                / (all_den_clip_da.sel(band=4) + all_den_clip_da.sel(band=1))
            )
            
            # Accumulate result
            all_den_tile_das.append(all_den_ndvi_da)

        # Merge data
        all_den_scene_da = merge_arrays(all_den_tile_das)

        # Mask vegetation
        all_den_veg_mask = (all_den_scene_da>.3)

        # Calculate statistics and save data to file
        # Calculate total number of non-missing (valid) pixels in the merged raster
        total_pixels = all_den_scene_da.notnull().sum()
        # Calculates total number of pixels that are classified as vegetation
        veg_pixels = all_den_veg_mask.sum()

        # Calculate mean patch size
        # Label the connected areas
        all_labeled_patches, all_num_patches = label(all_den_veg_mask)
        # Count patch pixels, ignoring background at patch 0
        all_patch_sizes = np.bincount(all_labeled_patches.ravel())[1:] 
        # Get the mean 
        all_mean_patch_size = all_patch_sizes.mean()

        # Calculate edge density
        all_kernel = np.array([
            [1, 1, 1], 
            [1, -8, 1], 
            [1, 1, 1]])
        
        # Apply convolution to the vegetation mask
        all_den_edges = convolve(
            # Input array - the array to apply convolution on
            all_den_veg_mask, 
            # Kernel array - the smaller array that defines the filter to be applied
            all_kernel, 
            # Input array is extended beyond its boundaries by - 
            # filling all values beyond the edge with the same constant value
            mode='constant')

        # Calculate edge density = 
        # count of edge pixels present / total number of pixels in the veg_mask
        all_den_edge_density = np.sum(all_den_edges != 0) / all_den_veg_mask.size

        # Add a row to the statistics file for this tract
        pd.DataFrame(dict(
            # Unique identifier for a given tract
            tract=[tract],
            # Cast totall number of pixels to an integer
            total_pixels=[int(total_pixels)],
            # Cast the fraction of pixels in the tract that are vegetation to a float
            frac_veg=[float(veg_pixels/total_pixels)],
            # Mean patch size of vegetation for this tract,
            all_mean_patch_size=[all_mean_patch_size],
            # Edge density of vegetation for this tract
            all_den_edge_density=[all_den_edge_density]
            # Write the df to a csv file
        )).to_csv(
            # The file path where the CSV will be saved to
            den_ndvi_stats_path, 
            # Ensure that the data is appended to the file rather than overwriting it
            mode='a', 
            # Prevent the index from being written to the CSV file
            index=False, 
            # Check if the file exists
            header=(not os.path.exists(den_ndvi_stats_path))
        )

# Re-load results from file
all_den_ndvi_stats_df = pd.read_csv(den_ndvi_stats_path)
# Call this df to see it
all_den_ndvi_stats_df

In [ ]:
# Store variables to use in next notebook
%store all_den_ndvi_stats_df 